In [51]:
import concurrent.futures
import threading
import psycopg2

lock = threading.Lock()


  

def update_site_locking(domain, sitemap, robotstxt):
    conn = psycopg2.connect(host="localhost", user="user", password="SecretPassword")
    conn.autocommit = True
    
    with lock:
        cur = conn.cursor()
        cur.execute("INSERT INTO site (domain, robots_content,sitemap_content) VALUES (%s, %s, %s) RETURNING id;",
        (domain, sitemap, robotstxt))
        id = cur.fetchone()[0]

    conn.commit()
    cur.close()
    conn.close()
                    
    return id;
                    
def update_page_locking(siteId, url,html_content, status_code, acc_time, page_type_code):
    conn = psycopg2.connect(host="localhost", user="user", password="SecretPassword")
    conn.autocommit = True
    
    with lock:
        cur = conn.cursor()
        cur.execute("INSERT INTO page (site_id, url,html_content,http_status_code,accessed_time, page_type_code) VALUES (%s,%s,%s,%s,%s,%s) RETURNING id;",
        (siteId, url, html_content,status_code,acc_time,page_type_code))
        id = cur.fetchone()[0]

    conn.commit()
    cur.close()
    conn.close()
                    
    return id;
                    
def update_image_locking(pageId, filename, content_type, data, acc_time):
    conn = psycopg2.connect(host="localhost", user="user", password="SecretPassword")
    conn.autocommit = True
    
    with lock:
        cur = conn.cursor()
        cur.execute("INSERT INTO image (page_id,filename, content_type,data,accessed_time) VALUES (%s,%s,%s, %s, %s) RETURNING id;",
        (pageId, filename, content_type,data,acc_time))
        id = cur.fetchone()[0]

    conn.commit()
    cur.close()
    conn.close()
                    
    return id;
                    
def update_page_data_locking(pageId, data_type_code, data):
    conn = psycopg2.connect(host="localhost", user="user", password="SecretPassword")
    conn.autocommit = True
    
    with lock:
        cur = conn.cursor()
        cur.execute("INSERT INTO page_data (page_id,data_type_code, data) VALUES (%s, %s, %s) RETURNING id;",
        (pageId, data_type_code, data))
        id = cur.fetchone()[0]

    conn.commit()
    cur.close()
    conn.close()
                    
    return id;
                    
                                        
def update_link_locking(from_page, to_page):
    conn = psycopg2.connect(host="localhost", user="user", password="SecretPassword")
    conn.autocommit = True
    
    with lock:
        cur = conn.cursor()
        cur.execute("INSERT INTO link (from_page,to_page) VALUES (%s, %s);",
        (from_page, to_page))

    conn.commit()
    cur.close()
    conn.close()
                    



In [43]:
import requests
from lxml import html
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time
import queue
import re
from urllib.parse import urljoin
from urllib.parse import urlparse


frontier = queue.Queue()
WEB_DRIVER_LOCATION = "C:/Users/Pirk/Desktop/faks-mag/ekstrakcija/chromedriver.exe"
TIMEOUT = 5
url =  "https://www.gov.si/"

def is_absolute(link):
    return bool(urlparse(link).netloc)

def is_link(link):
    if (len(re.findall('.:\/\/.',link))>0):
        return True
    return False
    #logika - preveri če je stvar valid link ki ga damo v frontier
    
def make_absolute(baselink,link):
    if (True != is_absolute(link)):
        return(urljoin(baselink,link))
    elif(link[0] == '/' and link[1] == '/'):
        return 'https:' + link
    return link
    #logika - prepozna če je link relativen inga spremeni v absolutnega

def get_robotstxt(link):
    chrome_options = Options()
    chrome_options.add_argument("user-agent=fri-ieps-TEST")
    driver = webdriver.Chrome(WEB_DRIVER_LOCATION, options=chrome_options)
    driver.get(link + '/robots.txt')
    time.sleep(TIMEOUT)
    html = driver.page_source
    bsObj = BeautifulSoup(html, 'html.parser')
    robots = bsObj.find('pre').contents
    return robots

def get_sitemap(robotstxt):
    robotarray = robotstxt.split()
    print((robotarray))

    print(robotarray)
    if("Sitemap:" in robotarray):
        index = robotarray.index("Sitemap:")
        return robotarray[index+1]
    return None
    
    
def crawl_site(url):

    WEB_PAGE_ADDRESS = url
    baseurl = re.match('^.+?[^\/:](?=[?\/]|$)', url).group(0)
    print(baseurl)
    
    chrome_options = Options()
    chrome_options.add_argument("user-agent=fri-ieps-TEST")
    driver = webdriver.Chrome(WEB_DRIVER_LOCATION, options=chrome_options)
    driver.get(WEB_PAGE_ADDRESS)
    time.sleep(TIMEOUT)

    html = driver.page_source

    bsObj = BeautifulSoup(html, 'html.parser')
    links = bsObj.find_all('a', href=True)
    #finalLinks = set()
    for link in links:
        if (len(link.attrs['href'])>0 and link.attrs['href'][0] != '#'):
            frontier.put(make_absolute(baseurl,link.attrs['href']))
            
    #print(list(frontier.queue))
    print("done1")

    domain = baseurl
    robotstxt = get_robotstxt(url)

    str1 = ''.join(robotstxt)


    sitemap = get_sitemap(str1)
    print(sitemap)
crawl_site(url)
    

https://www.gov.si
done1
['User-agent:', '*', 'Disallow:', '/admin', 'Disallow:', '/resources', 'Disallow:', '/pomoc', 'Sitemap:', 'https://www.gov.si/sitemap.xml']
['User-agent:', '*', 'Disallow:', '/admin', 'Disallow:', '/resources', 'Disallow:', '/pomoc', 'Sitemap:', 'https://www.gov.si/sitemap.xml']
https://www.gov.si/sitemap.xml
